# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
display(y.sample(5))

1018    1
561     0
368     0
1238    0
1572    0
Name: credit_ranking, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance

X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)


# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
n_inputs = X_train_scaled.shape[1]

# Review the number of features
print("Number of inputs (features):", n_inputs)



Number of inputs (features): 11


In [10]:
# Define the number of hidden nodes for the first hidden layer
n_hidden_nodes_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
n_hidden_nodes_layer2 = 32

# Define the number of neurons in the output layer
n_output_neurons = 1

In [11]:
# Create the Sequential model instance
model = Sequential()


# Add the first hidden layer
model.add(Dense(n_hidden_nodes_layer1, activation='relu', input_shape=(n_inputs,)))


# Add the second hidden layer
model.add(Dense(n_hidden_nodes_layer2, activation='relu'))


# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(n_output_neurons, activation='sigmoid'))


In [12]:
# Display the Sequential model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                768       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [14]:
# Fit the model using 50 epochs and the training data
fit_model = model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 4s 6ms/step - loss: 0.6404 - accuracy: 0.6397
Epoch 2/50
38/38 [==============================] - 0s 6ms/step - loss: 0.5421 - accuracy: 0.7323
Epoch 3/50
38/38 [==============================] - 0s 8ms/step - loss: 0.5132 - accuracy: 0.7515
Epoch 4/50
38/38 [==============================] - 0s 8ms/step - loss: 0.5006 - accuracy: 0.7606
Epoch 5/50
38/38 [==============================] - 0s 12ms/step - loss: 0.4908 - accuracy: 0.7665
Epoch 6/50
38/38 [==============================] - 1s 13ms/step - loss: 0.4825 - accuracy: 0.7706
Epoch 7/50
38/38 [==============================] - 0s 6ms/step - loss: 0.4770 - accuracy: 0.7773
Epoch 8/50
38/38 [==============================] - 0s 5ms/step - loss: 0.4707 - accuracy: 0.7756
Epoch 9/50
38/38 [==============================] - 0s 4ms/step - loss: 0.4626 - accuracy: 0.7823
Epoch 10/50
38/38 [==============================] - 0s 5ms/step - loss: 0.4587 - accuracy: 0.7832
Epoch 11/50
38/38

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data

model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


13/13 - 0s - loss: 0.5749 - accuracy: 0.7450 - 185ms/epoch - 14ms/step
Loss: 0.5748547911643982, Accuracy: 0.7450000047683716


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [27]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
model.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [28]:
# Set the model's file path
model_file_path = 'student_loans.keras'

from tensorflow.keras.models import load_model
# Load the model to a new object
loaded_model = load_model(model_file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [18]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)


# Display a sample of the predictions
print(predictions[:10])


13/13 [==============================] - 0s 2ms/step
[[0.13772926]
 [0.35486677]
 [0.8289462 ]
 [0.78660375]
 [0.9952754 ]
 [0.9674055 ]
 [0.98920804]
 [0.00599669]
 [0.8603177 ]
 [0.4634495 ]]


In [24]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions, columns=['prediction'])

predictions_df["prediction"] = round(predictions_df["prediction"],0).astype(int)

# Display the predictions DataFrame
predictions_df.head()

,prediction
0,0
1,0
2,1
3,1
4,1


### Step 4: Display a classification report with the y test data and predictions

In [20]:
# Print the classification report with the y test data and predictions

print(classification_report(y_test, predictions_df['prediction']))

              precision    recall  f1-score   support

           0       0.75      0.69      0.72       188
           1       0.74      0.79      0.77       212

    accuracy                           0.74       400
   macro avg       0.75      0.74      0.74       400
weighted avg       0.75      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**



## Student Load Recommendation System

---

### Data Collection


To build a recommendation system for student loans, I would collect the following data:



1.   **Student profiles:**  demographics, academic performance, financial situation, and career goals.
2.   **Loan options:** interest rates, repayment terms, fees, and lender information.
3.   **Historical data:**  loan applications, approvals, and repayment history.

This data is relevant and appropriate because it allows the system to understand the student's needs and financial situation, as well as the characteristics of the loan options. By analyzing historical data, the system can identify patterns and trends to make informed recommendations.

### Filtering Method:
Based on the data, I would use context-based filtering. This method considers the student's individual circumstances and preferences to recommend loan options that best fit their needs. The data I selected provides the necessary context to make personalized recommendations, such as:


*   Student demographics and financial situation to determine loan eligibility and affordability.
*   Academic performance and career goals to suggest loan options with favorable repayment terms.
*   Historical data to identify successful loan applications and repayment patterns.

### Real-World Challenges:
**Data Quality and Availability:** Ensuring access to accurate and up-to-date data on loan options and student profiles. This challenge is crucial because incomplete or inaccurate data can lead to biased or irrelevant recommendations.

**Regulatory Compliance:** Navigating complex regulations and laws governing student loans, such as disclosure requirements and lending standards. This challenge is essential because non-compliance can result in legal and reputational risks.

**Bias and Fairness:** Ensuring the recommendation system does not discriminate against certain groups of students (e.g., based on demographics or institution type).

**Data Security and Privacy:** Protecting sensitive student PII information

These challenges are critical in a student loan recommendation system, as they directly impact the system's effectiveness and trustworthiness. By addressing these challenges, the system can provide reliable and personalized recommendations that benefit students and lenders alike.